In [1]:
# Define the target directory
import os
from src.preprocessing.video import normalize_path

target_directory = "D:\\projects\\v2v\\v5\\tv360\\"

def has_exactly_two_mkv_files(directory):
    count = 0
    with os.scandir(directory) as entries:
        for entry in entries:
            # Check if the current entry is a file and ends with ".mkv"
            if entry.is_file() and (entry.name.endswith(".mkv") or entry.name.endswith(".mp4")):
                count += 1
                # Exit early if more than two such files are found
                if count > 2:
                    return False
    return count == 2

def list_end_directories(path):
    """
    Recursively traverse a directory and return all paths that are files (end of branch).
    """
    end_files = []
    for root, _, files in os.walk(path):
        for file in files:
            if file.startswith('SoccerNetV2'):                
                pass
            else:
                refined_path = normalize_path(os.path.join(root, file))
                # print(refined_path)
                # if has_exactly_two_mkv_files(refined_path):
                end_files.append(refined_path)
    end_files = ["/".join(file.split("/")[:-1]) for file in end_files]
    end_files = [item for item in end_files if has_exactly_two_mkv_files(item)] 
    end_files = sorted(list(set(end_files)))
    return end_files
end_files = list_end_directories(target_directory)
end_files[0]

'D:/projects/v2v/v5/tv360/gir_vs_feynord'

In [2]:

import os
from src.preprocessing.video import VideoProcessor, VideoProcessorConfig
import shutil
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from src.preprocessing.video import normalize_path

def process_soccernet(vid: str):
    config = VideoProcessorConfig(input_file = f"{vid}/1_224p.mp4",
                                segment_duration = 60,
                                base_output_dir= 'D:/projects/v2v/v5/tv360/video_classification',
                                output_dir = '1_224p.mp4',
                                session = 'first_half',
                                overlapping = 30, 
                                frame_interval = 1)
    video_splitter = VideoProcessor(config = config)
    video_splitter.chunk_video(do_sep = True)
    game_name = vid.split("/")[-1]
    new_game_path = os.path.join(config.base_output_dir, game_name)
    game_chunk_paths = sorted(
        [
            os.path.join(new_game_path, i, 'frames')
            for i in os.listdir(new_game_path)
            if not i.endswith('.json')
        ]
    )
    for item in game_chunk_paths:
        if len(os.listdir(item)) <10:
            parent_dir = os.path.dirname(item)
            shutil.rmtree(parent_dir)

    return new_game_path

vidv2 = end_files[0]
process_soccernet(normalize_path(vidv2))


In [5]:
from typing import List
import glob
from src.models.video import TimeSformerInferencing
from src.post_processing import process_football_events, cfg
from src.post_processing.utils import concat_videos
from src.preprocessing.video import normalize_path

from src.models.video import TimeSformerInferencing
from src.post_processing.utils import read_json, joint_event
from typing import List
inferencing =TimeSformerInferencing(
    num_frames= 30,
    num_classes = 17,
    game_name = 'tv360/video_classification/gir_vs_feynord',
    image_extractor="fcakyon/timesformer-large-finetuned-k400",
    checkpoint = './ds_tv360_model_timesformer_nepochs_2_nframes_30.pth')

results = inferencing.run(method='from_middle', spacing = 30)

Some weights of TimesformerModel were not initialized from the model checkpoint at fcakyon/timesformer-large-finetuned-k400 and are newly initialized because the shapes did not match:
- timesformer.embeddings.time_embeddings: found shape torch.Size([1, 96, 768]) in the checkpoint and torch.Size([1, 30, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 99/99 [02:23<00:00,  1.45s/it]


In [6]:
results

'gir_vs_feynord_prediction.jsonl'

In [7]:
game, event2timestamp = process_football_events(json_path = "./gir_vs_feynord_prediction.jsonl", rank = cfg.rank)

In [9]:
event2timestamp

{'Goal': [(22, 23),
  (35, 36),
  (50, 51),
  (68, 69),
  (75, 76),
  (80, 81),
  (88, 89),
  (97, 98)],
 'Penalty': [],
 'Cards': [62],
 'Foul': [5, 25],
 'Substitution': [(24, 26), (77, 78)],
 'Shots': [9, (17, 18), 46, 67, 79]}

In [15]:
basedir = "D:/projects/v2v/v5/tv360/video_classification"
game_path = f"{basedir}/{game}"
for event, timestamps in event2timestamp.items():
    for timestamp in timestamps:
        if isinstance(timestamp, tuple):
            print(glob.glob(f"{game_path}/chunk_{timestamp[0]}/*.mp4"))
            video1 = normalize_path(glob.glob(f"{game_path}/chunk_{timestamp[0]}/*.mp4")[0])
            video2 =  normalize_path(glob.glob(f"{game_path}/chunk_{timestamp[1]}/*.mp4")[0])
            
            concat_videos([video1, video2], f"{event}_{timestamp[0]}_{timestamp[1]}.mp4")
        elif isinstance(timestamp, int):
            video = normalize_path(glob.glob(f"{game_path}/chunk_{timestamp}/*.mp4")[0])
            concat_videos([video], f"{event}_{timestamp}.mp4")
        else:
            pass

['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_22\\video_22_start_1320_end_1380.mp4']
Đã nối video thành công: Goal_22_23.mp4
['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_35\\video_35_start_2100_end_2160.mp4']
Đã nối video thành công: Goal_35_36.mp4
['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_50\\video_50_start_168_end_228.mp4']
Đã nối video thành công: Goal_50_51.mp4
['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_68\\video_68_start_1248_end_1308.mp4']
Đã nối video thành công: Goal_68_69.mp4
['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_75\\video_75_start_1668_end_1728.mp4']
Đã nối video thành công: Goal_75_76.mp4
['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_80\\video_80_start_1968_end_2028.mp4']
Đã nối video thành công: Goal_80_81.mp4
['D:/projects/v2v/v5/tv360/video_classification/gir_vs_feynord/chunk_88\\video_88_start_2448_end_2508.mp4']
Đã nối

In [19]:
import subprocess
import os
import glob

def resize_video(input_path, output_path, height=224):
    """Resize a single video file"""
    try:
        # Ensure the output directory exists
        os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
        
        ffmpeg_cmd = [
            'ffmpeg',
            '-i', input_path,
            '-vf', f'scale=-1:{height}',
            '-c:v', 'libx264',
            '-crf', '18',
            '-preset', 'medium',
            '-c:a', 'copy',
            output_path
        ]
        
        process = subprocess.run(
            ffmpeg_cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        
        if process.returncode != 0:
            print(f"Error processing {input_path}: {process.stderr}")
            return False
        
        print(f"Successfully resized {input_path} to {height}p")
        return True
        
    except Exception as e:
        print(f"An error occurred with {input_path}: {str(e)}")
        return False

def batch_resize_videos(input_dir, output_dir, pattern="*.mp4", height=224):
    """Resize all videos matching pattern in input_dir and save to output_dir"""
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Find all matching video files
    video_files = glob.glob(os.path.join(input_dir, pattern))
    
    if not video_files:
        print(f"No files matching {pattern} found in {input_dir}")
        return
    
    print(f"Found {len(video_files)} videos to process")
    
    # Process each video
    success_count = 0
    for input_path in video_files:
        filename = os.path.basename(input_path)
        output_filename = f"{os.path.splitext(filename)[0]}_{height}p{os.path.splitext(filename)[1]}"
        output_path = os.path.join(output_dir, output_filename)
        
        if resize_video(input_path, output_path, height):
            success_count += 1
    
    print(f"Completed: {success_count}/{len(video_files)} videos successfully resized")
    
filelist = [f"D:/projects/v2v/v5/tv360/result/{i}" for i in os.listdir("D:/projects/v2v/v5/tv360/result")]
for video in tqdm(filelist):
    output_video = os.path.join(os.path.dirname(video), 'tempt', f"{video.split("/")[-1]}")
    resize_video(video, output_video)

  5%|▌         | 1/19 [00:02<00:44,  2.47s/it]

Successfully resized D:/projects/v2v/v5/tv360/result/Cards_62.mp4 to 224p


 11%|█         | 2/19 [00:04<00:42,  2.47s/it]

Successfully resized D:/projects/v2v/v5/tv360/result/Foul_25.mp4 to 224p


 16%|█▌        | 3/19 [00:07<00:39,  2.45s/it]

...

Successfully resized D:/projects/v2v/v5/tv360/result/Shots_9.mp4 to 224p


 89%|████████▉ | 17/19 [01:06<00:07,  3.59s/it]

Successfully resized D:/projects/v2v/v5/tv360/result/Substitution_24_26.mp4 to 224p


100%|██████████| 19/19 [01:11<00:00,  3.74s/it]

Successfully resized D:/projects/v2v/v5/tv360/result/Substitution_77_78.mp4 to 224p
Error processing D:/projects/v2v/v5/tv360/result/tempt: ffmpeg version 7.1-essentials_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 14.2.0 (Rev1, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-zlib --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-sdl2 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxvid --enable-libaom --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-libfreetype --enable-libfribidi --enable-libharfbuzz --enable-libvidstab --enable-libvmaf --enable-libzimg --enable-amf --enable-cuda-llvm --enable-cuvid --enable-dxva2 --enable-d3d11va --enable-d3d12va --enable-ffnvcodec --enabl

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import skvideo.measure
import cv2
import skvideo.io
import os
from tqdm import tqdm
from src.post_processing.utils import filter_highlight_scenes, cut_video_by_frame
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool 

filelist = sorted([f"D:/projects/v2v/v5/tv360/result/{i}" for i in os.listdir("D:/projects/v2v/v5/tv360/result")])
for filename in tqdm(filelist):
    videodata = skvideo.io.vread(filename)
    # videometadata = skvideo.io.ffprobe(filename)
    scene_lum_idx = skvideo.measure.scenedet(videodata, method='histogram', parameter1=1.0)
    
    frame_cut = filter_highlight_scenes(scene_lum_idx)
    cut_video_by_frame(video_path = filename, frame_position = frame_cut, middle_threshold= 0.2)

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import skvideo.measure
import cv2
import skvideo.io
import os
from tqdm import tqdm
from src.post_processing.utils import filter_highlight_scenes, cut_video_by_frame
np.float = float    
np.int = int   #module 'numpy' has no attribute 'int'
np.object = object    #module 'numpy' has no attribute 'object'
np.bool = bool 

filelist = sorted([f"D:/projects/v2v/v5/tv360/result/{i}" for i in os.listdir("D:/projects/v2v/v5/tv360/result") if not os.path.isdir(f"D:/projects/v2v/v5/tv360/result/{i}")])
filelist2 = sorted([f"D:/projects/v2v/v5/tv360/result/tempt/{i}" for i in os.listdir("D:/projects/v2v/v5/tv360/result/tempt")])

for i, j in tqdm(zip(filelist, filelist2)):
    videodata = skvideo.io.vread(j)
    scene_lum_idx = skvideo.measure.scenedet(videodata, method='histogram', parameter1= 1.0)
    frame_cut = filter_highlight_scenes(scene_lum_idx)
    cut_video_by_frame(video_path = i, frame_position = frame_cut, middle_threshold= 0.1)

0it [00:00, ?it/s]

0
627


1it [00:04,  4.12s/it]

Frame 313 nằm ở nửa đầu video, giữ lại phần sau từ 10.44s. Kết quả: D:/projects/v2v/v5/tv360/result\Cards_62_second_half.mp4
670
1294


2it [00:08,  4.18s/it]

...

Frame 1027 nằm ở giữa video (frame 720-1080), giữ nguyên video. Kết quả: D:/projects/v2v/v5/tv360/result\Shots_9_full.mp4
509
1800


17it [01:37,  5.06s/it]

Frame 1154 nằm ở nửa đầu video, giữ lại phần sau từ 38.47s. Kết quả: D:/projects/v2v/v5/tv360/result\Substitution_24_26_second_half.mp4
153
1079


18it [01:45,  5.84s/it]

Frame 616 nằm ở nửa đầu video, giữ lại phần sau từ 20.54s. Kết quả: D:/projects/v2v/v5/tv360/result\Substitution_77_78_second_half.mp4


In [2]:
from src.preprocessing.video import normalize_path
basedir = "D:\\projects\\v2v\\v5\\tv360\\result\\correct"
[normalize_path(f"{basedir}/{i}") for i in os.listdir(basedir)]

['D:/projects/v2v/v5/tv360/result/correct/Cards_62_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Foul_25_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Foul_5_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_22_23_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_35_36_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_50_51_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_68_69_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_75_76_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_80_81_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_88_89_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_97_98_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_17_18_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_46_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_67_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_79_full.mp4',
 'D

In [3]:
import os
import subprocess
from pathlib import Path

def merge_videos(video_list, output_path, method="concat"):
    """
    Ghép nối nhiều video theo thứ tự trong danh sách.
    
    Args:
        video_list (list): Danh sách đường dẫn đến các video cần ghép nối
        output_path (str): Đường dẫn đến file video đầu ra
        method (str): Phương pháp ghép nối, "concat" hoặc "filter_complex"
    
    Returns:
        str: Đường dẫn đến video đã ghép nối
    """
    if not video_list:
        raise ValueError("Danh sách video trống!")
    
    # Kiểm tra tất cả các video có tồn tại không
    for video in video_list:
        if not os.path.exists(video):
            raise FileNotFoundError(f"Không tìm thấy file video: {video}")
    
    if method == "concat":
        # Phương pháp 1: Sử dụng file danh sách (tốt nhất cho video cùng codec)
        filelist_path = "temp_filelist.txt"
        
        with open(filelist_path, "w", encoding="utf-8") as f:
            for video in video_list:
                # Sử dụng đường dẫn tuyệt đối và escape ký tự đặc biệt
                abs_path = os.path.abspath(video).replace("\\", "/")
                f.write(f"file '{abs_path}'\n")
        
        cmd = f'ffmpeg -f concat -safe 0 -i "{filelist_path}" -c copy "{output_path}"'
        
        try:
            subprocess.run(cmd, shell=True, check=True)
            print(f"Đã ghép nối {len(video_list)} video thành công! Kết quả: {output_path}")
        except subprocess.CalledProcessError as e:
            print(f"Lỗi khi ghép nối video: {e}")
            return None
        finally:
            # Xóa file tạm
            if os.path.exists(filelist_path):
                os.remove(filelist_path)
    
    elif method == "filter_complex":
        # Phương pháp 2: Sử dụng filter_complex (cho video khác codec)
        inputs = ""
        filter_complex = ""
        
        for i, video in enumerate(video_list):
            inputs += f' -i "{video}"'
            filter_complex += f"[{i}:v:0][{i}:a:0]"
        
        filter_complex += f"concat=n={len(video_list)}:v=1:a=1[outv][outa]"
        cmd = f'ffmpeg{inputs} -filter_complex "{filter_complex}" -map "[outv]" -map "[outa]" "{output_path}"'
        
        try:
            subprocess.run(cmd, shell=True, check=True)
            print(f"Đã ghép nối {len(video_list)} video thành công! Kết quả: {output_path}")
        except subprocess.CalledProcessError as e:
            print(f"Lỗi khi ghép nối video: {e}")
            return None
    
    else:
        raise ValueError("Phương pháp không hợp lệ! Chỉ hỗ trợ 'concat' hoặc 'filter_complex'")
    
    return output_path

# Ví dụ sử dụng
video_list = ['D:/projects/v2v/v5/tv360/result/correct/Foul_5_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_9_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_17_18_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_22_23_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Substitution_24_26_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Foul_25_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_35_36_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_46_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_50_51_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Cards_62_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_67_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_68_69_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_75_76_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Substitution_77_78_second_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Shots_79_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_80_81_full.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_88_89_first_half.mp4',
 'D:/projects/v2v/v5/tv360/result/correct/Goal_97_98_second_half.mp4']

# Ghép nối sử dụng phương pháp concat (nhanh, giữ nguyên chất lượng)
merged_video = merge_videos(video_list, "tv360_sample.mp4", method="concat")

# Nếu có lỗi với phương pháp concat (ví dụ: codec khác nhau), thử phương pháp filter_complex
if not merged_video:
    merge_videos(video_list, "output_merged.mp4", method="filter_complex")


Đã ghép nối 18 video thành công! Kết quả: tv360_sample.mp4


In [10]:
import os
filelist = sorted([f"D:/projects/v2v/v5/tv360/result/{i}" for i in os.listdir("D:/projects/v2v/v5/tv360/result") if not os.path.isdir(f"D:/projects/v2v/v5/tv360/result/{i}")])
filelist2 = sorted([f"D:/projects/v2v/v5/tv360/result/tempt/{i}" for i in os.listdir("D:/projects/v2v/v5/tv360/result/tempt")])
len(filelist2)

18

In [6]:
os.path.isdir('D:/projects/v2v/v5/tv360/result/Substitution_77_78.mp4')

False

In [6]:
skvideo.measure.scenedet(videodata, method='histogram', parameter1=1.0)

array([   0,    4,    8,   12,   16,   20,   30,   53,   59,   63,   68,
         72,   77,   81,   86,   90,   94,   98,  102,  106,  110,  120,
        150,  154,  162,  166,  171,  175,  180,  184,  193,  214,  238,
        243,  247,  252,  256,  260,  266,  270,  274,  283,  290,  299,
        305,  320,  324,  328,  332,  336,  340,  344,  348,  352,  356,
        360,  365,  369,  373,  377,  381,  385,  389,  393,  398,  402,
        406,  410,  414,  418,  422,  426,  430,  434,  438,  442,  446,
        450,  454,  458,  462,  466,  470,  474,  478,  482,  486,  490,
        494,  498,  502,  506,  510,  514,  518,  522,  526,  530,  534,
        538,  542,  546,  550,  554,  558,  562,  566,  570,  574,  578,
        582,  586,  590,  594,  598,  602,  606,  610,  614,  618,  622,
        626,  735,  740,  749,  754,  763,  773,  780,  785,  809,  815,
        820,  824,  829,  840,  845,  869,  877,  881,  885,  892,  900,
        910,  914,  918,  922,  929,  933,  938,  9